In [1]:
import pandas as pd
import json
import requests
import time
import re
from bs4 import BeautifulSoup

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}

In [20]:
craigslist = pd.read_csv("craiglistOutput.csv")
craigslist = craigslist.sort_values('CarYear', ascending=False)
craigslist.head()

,CarMake,CarModel,CarYear,CarPrice,CarCondition,MilesOnCar,PostingDate,PostingUrl,CarImageLink,PostingRegion
444,Mitsubishi,Eclipse Cross,2022,22998.0,new,10,2021-05-23,https://kpr.craigslist.org/ctd/d/kennewick-202...,https://images.craigslist.org/00W0W_90ubo4k04K...,Kpr
227,Mitsubishi,Outlander SEL,2022,459.0,excellent,10,2021-05-21,https://bellingham.craigslist.org/ctd/d/spokan...,https://images.craigslist.org/00505_571ciMXtXg...,Bellingham
3211,Mitsubishi,Outlander SEL,2022,472.0,excellent,4,2021-05-18,https://wenatchee.craigslist.org/ctd/d/spokane...,https://images.craigslist.org/00Q0Q_3NSGPoZo8j...,Wenatchee
703,Nissan,Sentra SR,2021,25505.0,excellent,5,2021-05-21,https://kpr.craigslist.org/ctd/d/college-place...,https://images.craigslist.org/00z0z_3J4oABMmG1...,Kpr
2813,Jeep,Grand Cherokee,2021,74977.0,good,66,2021-05-22,https://spokane.craigslist.org/ctd/d/puyallup-...,https://images.craigslist.org/00000_3WomnsErfv...,Spokane


In [21]:
# all unique makes
makes = craigslist.CarMake.unique()
makes

array(['Mitsubishi', 'Nissan', 'Jeep', 'Chevrolet', 'Toyota', nan,
       'Volkswagen', 'Kia', 'Gmc', 'Dodge', 'Honda', 'Ford', 'Subaru',
       'Cadillac', 'Hyundai', 'Ram', 'Chrysler', 'Porsche', 'Mazda',
       'Bmw', 'Buick', 'Land Rover', 'Acura', 'Audi', 'Genesis', 'Lexus',
       'Fiat', 'Tesla', 'Infiniti', 'Alfa Romeo', 'Mini', 'Smart',
       'Jaguar', 'Lincoln', 'Aston Martin', 'Volvo', 'Scion', 'Mercedes',
       'Ferrari', 'Mercury', 'Maserati', 'Pontiac', 'Bentley', 'Hummer',
       'Saab', 'Saturn', 'Isuzu', 'Oldsmobile', 'Suzuki', 'Datsun',
       'Plymouth', 'Packard'], dtype=object)

In [22]:
models = craigslist.CarModel.unique()
models

array(['Eclipse Cross', 'Outlander SEL', 'Sentra SR', ...,
       'special deluxe', 'truck', 'Model A'], dtype=object)

In [23]:
years = craigslist.CarYear.unique()
years

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012,
       2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001,
       2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990,
       1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979,
       1978, 1977, 1975, 1974, 1973, 1972, 1970, 1969, 1968, 1966, 1965,
       1964, 1959, 1958, 1957, 1956, 1955, 1953, 1952, 1951, 1950, 1949,
       1948, 1946, 1942, 1941, 1940, 1938, 1934, 1930,   -1], dtype=int64)

In [24]:
make_model_year = {}
for make in makes:
    make_model_year[make] = {}
    
    craigslist_slice = craigslist.loc[craigslist.CarMake == make, ["CarModel", "CarYear"]]
    
    for year in craigslist_slice.CarYear.unique():
        temp = craigslist_slice.loc[craigslist.CarYear == year, "CarModel"].values
        make_model_year[make][year] =temp

In [27]:
make_model_year["Mitsubishi"]

{2022: array(['Eclipse Cross', 'Outlander SEL', 'Outlander SEL'], dtype=object),
 2021: array(['Outlander Sport SE', 'Outlander PHEV LE'], dtype=object),
 2020: array(['Outlander', 'Outlander Sport', 'Outlander', 'Mirage',
        'Outlander SEL 4x4 gas', 'Outlander SEL 4x4 gas',
        'Outlander SEL 4x4 gas'], dtype=object),
 2019: array(['mirage'], dtype=object),
 2017: array(['Lancer', 'MIRAGE G4'], dtype=object),
 2014: array(['OUTLANDER SPORT'], dtype=object),
 2013: array(['Lancer Evolution'], dtype=object),
 2012: array(['outlander sport es 4d'], dtype=object),
 2008: array(['LANCER', 'LANCER'], dtype=object),
 2006: array(['eclipse gs 2dr hatchb', 'lancer'], dtype=object),
 2004: array(['Endeavor'], dtype=object),
 2003: array(['lancer'], dtype=object)}

In [28]:
req = requests.get("https://www.carqueryapi.com/api/0.3/?callback=?&cmd=getModels&make=mitsubishi&year=2021", headers=headers).content

In [76]:
import difflib
def find_name(craigslist_names, database_names, year):
    ans = []
    
    for name in craigslist_names:
        name_2 = str(name).capitalize()
        guess = difflib.get_close_matches(name_2, database_names, n=1, cutoff=.5)
        if len(guess) == 0:
            guess = ["NA"]
        print((name, guess[0], year))
        ans.append((name, guess[0], year))
    return ans

In [78]:
url = "https://www.carqueryapi.com/api/0.3/?callback=?&cmd=getModels&make={}&year={}"
answer = []
for make in make_model_year:
    print(make)
    for year in make_model_year[make]:
        if year > 2021 or year < 1941:
            continue
        # ordered like so: 
        # dict[make][year] = [(model raw, model db)]
        models = make_model_year[make][year]
        req = requests.get(url.format(make,year), headers=headers).content
        req = json.loads(req[2:-2])
        model_names = []
        for model_name in req["Models"]:
            model_names.append(model_name["model_name"])
        answer.extend(find_name(models, model_names, year))
        time.sleep(1)
answer

Mitsubishi
('Outlander Sport SE', 'Outlander Sport', 2021)
('Outlander PHEV LE', 'Outlander', 2021)
('Outlander', 'Outlander', 2020)
('Outlander Sport', 'Outlander Sport', 2020)
('Outlander', 'Outlander', 2020)
('Mirage', 'Mirage', 2020)
('Outlander SEL 4x4 gas', 'Outlander', 2020)
('Outlander SEL 4x4 gas', 'Outlander', 2020)
('Outlander SEL 4x4 gas', 'Outlander', 2020)
('mirage', 'Mirage', 2019)
('Lancer', 'Lancer', 2017)
('MIRAGE G4', 'Mirage', 2017)
('OUTLANDER SPORT', 'Outlander Sport', 2014)
('Lancer Evolution', 'Lancer', 2013)
('outlander sport es 4d', 'Outlander', 2012)
('LANCER', 'Lancer', 2008)
('LANCER', 'Lancer', 2008)
('eclipse gs 2dr hatchb', 'Eclipse', 2006)
('lancer', 'Lancer', 2006)
('Endeavor', 'Endeavor', 2004)
('lancer', 'Lancer', 2003)
Nissan
('Sentra SR', 'Sentra', 2021)
('Armada SV', 'Armada', 2021)
('Murano Platinum', 'Murano', 2021)
('Altima 2.5 SR', 'Altima', 2021)
('Murano Platinum', 'Murano', 2021)
('Altima 2.5 SV', 'Altima', 2021)
('Murano Platinum', 'Murano

('WRANGLER', 'Wrangler', 2017)
('Renegade Latitude 4x4 gas', 'NA', 2017)
('Grand Cherokee Limited 4x4', 'Grand Cherokee', 2017)
('Grand Cherokee Laredo 4WD', 'Grand Cherokee', 2017)
('Renegade', 'Renegade', 2017)
('Wrangler Unlimited Sport', 'Wrangler', 2017)
('Wrangler Unlimited', 'Wrangler', 2017)
('Renegade Altitude 4x4 gas', 'NA', 2017)
('renegade latitude 4x4 gas', 'NA', 2017)
('Cherokee Limited 4x4 gas', 'Cherokee', 2017)
('Grand Cherokee SRT-8', 'Grand Cherokee', 2017)
('Wrangler Unlimited Sahara', 'NA', 2017)
('Renegade', 'Renegade', 2017)
('Wrangler Unlimited Smoky', 'Wrangler', 2017)
('WRANGLER', 'Wrangler', 2017)
('Grand Cherokee Laredo 4WD', 'Grand Cherokee', 2017)
('WRANGLER', 'Wrangler', 2017)
('Grand Cherokee Laredo 4WD', 'Grand Cherokee', 2017)
('Grand Cherokee', 'Grand Cherokee', 2017)
('Grand Cherokee Laredo 4WD', 'Grand Cherokee', 2017)
('Wrangler Unlimited Sport', 'Wrangler', 2017)
('Wrangler Unlimited Rubicon', 'NA', 2017)
('WRANGLER', 'Wrangler', 2017)
('Grand Che

('Camaro LT Convertible', 'NA', 2019)
('Silverado Diesels 4x4', 'Silverado 1500', 2019)
('Silverado 1500 LT Boss', 'Silverado 1500', 2019)
('Silverado 1500 Custom', 'Silverado 1500', 2019)
('Silverado 1500 Regular', 'Silverado 1500', 2019)
('TRAVERSE PREMIER', 'Traverse', 2019)
('Silverado 1500 RST 4x4', 'Silverado 1500', 2019)
('Suburban 1500 Premier', 'Suburban', 2019)
('Silverado 1500 Regular', 'Silverado 1500', 2019)
('Silverado Diesels 3500', 'Silverado 3500HD', 2019)
('silverado 2500hd', 'Silverado 2500HD', 2019)
('Camaro LT Convertible', 'NA', 2019)
('Silverado 1500', 'Silverado 1500', 2019)
('Silverado 1500 Regular', 'Silverado 1500', 2019)
('Silverado 1500 Custom', 'Silverado 1500', 2019)
('Traverse Premier AWD', 'Traverse', 2019)
('Silverado Diesel 3500', 'Silverado 3500HD', 2019)
('Silverado 3500HD', 'Silverado 3500HD', 2019)
('Camaro LT Convertible', 'NA', 2019)
('Silverado 1500 LT 4x4', 'Silverado 1500', 2019)
('Silverado 1500', 'Silverado 1500', 2019)
('Silverado 1500', '

('Equinox', 'Equinox', 2011)
('Tahoe LT Sport Utility', 'NA', 2011)
('Camaro', 'Camaro', 2011)
('Traverse', 'Traverse', 2011)
('Cruze LTZ', 'Cruze', 2011)
('Equinox', 'Equinox', 2011)
('Camaro', 'Camaro', 2011)
('equinox ls', 'Equinox', 2011)
('CRUZE', 'Cruze', 2011)
('CRUZE LT', 'Cruze', 2011)
('SILVERADO', 'Silverado', 2011)
('Tahoe LT Sport Utility', 'NA', 2011)
('Camaro', 'Camaro', 2011)
('Silverado Diesels 2500', 'Silverado', 2011)
('Impala', 'Impala', 2011)
('Silverado 2500HD', 'Silverado', 2011)
('CRUZE', 'Cruze', 2011)
('SILVERADO', 'Silverado', 2011)
('Traverse', 'Traverse', 2011)
('Traverse', 'Traverse', 2010)
('EQUINOX', 'Equinox', 2010)
('IMPALA', 'Impala', 2010)
('Malibu HY', 'Malibu', 2010)
('Traverse', 'Traverse', 2010)
('express 3500 extended', 'Express', 2010)
('CAMARO', 'Camaro', 2010)
('Equinox', 'Equinox', 2009)
('Tahoe LTZ', 'Tahoe', 2009)
('2500HD', 'NA', 2009)
('Aveo5 1LT', 'Aveo', 2009)
('traverse', 'Traverse', 2009)
('Silverado 3500HD', 'Silverado', 2009)
('sil

('4Runner SR5 Sport Utility', 'NA', 2013)
('Corolla', 'Corolla', 2013)
('PRIUS V', 'Prius', 2013)
('4Runner SR5 Sport Utility', 'NA', 2013)
('tacoma double cab sr5', 'NA', 2013)
('Tacoma V6', 'Tacoma', 2013)
('Tacoma', 'Tacoma', 2013)
('Prius', 'Prius', 2013)
('Corolla', 'Corolla', 2013)
('FJ Cruiser 4x4 gas SUV', 'FJ Cruiser', 2013)
('4Runner SR5 Sport Utility', 'NA', 2013)
('Tacoma', 'Tacoma', 2013)
('Tacoma Double Cab', 'Tacoma', 2013)
('Tundra CrewMax Pickup 4D', 'NA', 2012)
('FJ Cruiser', 'FJ Cruiser', 2012)
('Tundra CrewMax Pickup 4D', 'NA', 2012)
('Tundra CrewMax Pickup 4D', 'NA', 2012)
('COROLLA', 'Corolla', 2012)
('Corolla L', 'Corolla', 2012)
('Sequoia', 'Sequoia', 2012)
('Tundra CrewMax Pickup 4D', 'NA', 2012)
('COROLLA', 'Corolla', 2012)
('Sequoia', 'Sequoia', 2012)
('TUNDRA', 'Tundra', 2012)
('COROLLA', 'Corolla', 2012)
('Tundra CrewMax Pickup 4D', 'NA', 2012)
('rav 4', 'NA', 2011)
('RAV4', 'RAV4', 2011)
('Tundra', 'Tundra', 2011)
('RAV4 Sport 4WD', 'NA', 2011)
('RAV4', 'R

...

('tucson', 'Tucson', 2012)
('Elantra', 'Elantra', 2012)
('SONATA', 'Sonata', 2012)
('Sonata Limited FWD gas', 'NA', 2012)
('SONATA', 'Sonata', 2012)
('ELANTRA', 'Elantra', 2011)
('sonata', 'Sonata', 2010)
('santa fe se', 'Santa Fe', 2009)
('elantra', 'Elantra', 2008)
('elantra', 'Elantra', 2006)
('tuscon v6', 'Tucson', 2006)
('santa fe gls', 'Santa Fe', 2004)
('santa fe', 'Santa Fe', 2001)
('Accent', 'NA', 1997)
Ram
('1500 Big Horn/Lone Star', 'NA', 2020)
('1500', '1500', 2020)
('1500 Big Horn/Lone Star', 'NA', 2020)
('2500', '2500', 2020)
('2500', '2500', 2020)
('2500', '2500', 2020)
('1500 Classic', '1500', 2020)
('2500', '2500', 2020)
('3500 Mega Cab SPORT', 'NA', 2020)
('3500 Mega Cab SPORT', 'NA', 2020)
('1500 Big Horn HEMI LOADED', 'NA', 2020)
('1500', '1500', 2020)
('1500 Classic', '1500', 2020)
('1500', '1500', 2019)
('1500 Classic Quad Cab', 'NA', 2019)
('1500 Rebel Crew Cab', 'NA', 2019)
('1500', '1500', 2019)
('1500 Classic Quad Cab', 'NA', 2019)
('1500 Classic Quad Cab', 'N

('M4', 'M4', 2018)
('2 Series 230i Convertible 2D', 'NA', 2018)
('3 Series 330i xDrive AWD', 'NA', 2018)
('328d xdrive', '3 Series', 2018)
('4 Series', '4 Series', 2018)
('3 Series', '3 Series', 2018)
('M3', 'M3', 2018)
('M3', 'M3', 2018)
('2 Series 230i Convertible 2D', 'NA', 2018)
('2 Series 230i Convertible 2D', 'NA', 2018)
('X5 M Base', 'NA', 2018)
('X1 XDRIVE28I AWD gas SUV', 'NA', 2017)
('X5 xDrive35i -- 3rd ROW SEA', 'NA', 2017)
('4 Series 440i xDrive Gran', '4 Series Gran Coupe', 2017)
('X5 xDrive35d -- Diesel w/ 3', 'NA', 2017)
('X5 xDrive35d', '5 Series', 2017)
('X5 xDrive35i', '5 Series', 2017)
('X3 xDrive28i M-Sport PKG', 'NA', 2017)
('4 Series 440i xDrive Gran', '4 Series Gran Coupe', 2017)
('4 Series 440i xDrive Gran', '4 Series Gran Coupe', 2017)
('4 Series 440i xDrive Gran', '4 Series Gran Coupe', 2017)
('X5 xDrive35i', '5 Series', 2017)
('6 Series', '6 Series', 2017)
('4 Series 440i xDrive Gran', '4 Series Gran Coupe', 2017)
('5 Series 540i xDrive', '5 Series Gran Turi

('MDX SH-AWD', 'NA', 2008)
('TSX w/Navi', 'NA', 2008)
('MDX SH-AWD', 'NA', 2007)
('MDX Touring w/Navi', 'NA', 2006)
('MDX', 'NA', 2003)
('3.2 tl', 'NA', 2003)
('3.2TL', 'NA', 2003)
('3.0cl', 'NA', 1997)
('Integra', 'Integra', 1996)
Audi
('Q5', 'Q5', 2019)
('Q8 3.0T quattro Prestige', 'NA', 2019)
('A4 Ultra Premium Sedan 4D', 'NA', 2018)
('Q3 Quattro AWD gas SUV Auto', 'NA', 2018)
('A4 Ultra Premium Sedan 4D', 'NA', 2018)
('Q3 Premium Plus AWD gas SUV', 'NA', 2018)
('A4 Premium Plus AWD gas', 'NA', 2018)
('A4 Ultra Premium Sedan 4D', 'NA', 2018)
('q7', 'Q7', 2018)
('Q5 Premium Sport Utility 4D', 'NA', 2018)
('Q5', 'Q5', 2018)
('S4', 'S4', 2018)
('S7', 'S7', 2018)
('A4 Ultra Premium Sedan 4D', 'NA', 2018)
('A6 Competition Quattro AWD', 'NA', 2017)
('A4', 'A4', 2017)
('A3 Sportback e-tron', 'A3 e-tron', 2016)
('S4 Premium Plus Sedan 4D', 'NA', 2016)
('S4 Premium Plus Sedan 4D', 'NA', 2016)
('SQ5', 'S5', 2015)
('SQ5', 'S5', 2015)
('SQ5', 'S5', 2015)
('Q7', 'Q7', 2015)
('allroad Premium Wag

Hummer
('H2 Luxury LSA SUPERCHARG', 'NA', 2008)
('H3 Base 4dr SUV 4WD', 'NA', 2006)
('H2', 'H2', 2003)
('H2 4x4 gas SUV Automatic', 'NA', 2003)
Saab
('9-5 2.3T', 'NA', 2008)
('9-3 Linear 124K MILES', 'NA', 2005)
('9E 900', '900', 1997)
('900 turbo convertible', 'NA', 1988)
Saturn
('AURA', 'Aura', 2007)
('vue', 'Vue', 2007)
('Ion', 'NA', 2006)
('Vue', 'Vue', 2003)
('vue', 'Vue', 2003)
Isuzu
('NPR', 'NA', 2007)
('rodeo 4x4', 'NA', 1993)
Oldsmobile
('silhouette', 'Silhouette', 2002)
('silhouette', 'Silhouette', 2002)
('BRAVADA', 'Bravada', 2001)
('silhouette premiere', 'Silhouette', 1999)
('88', 'NA', 1998)
('Super 88', 'NA', 1953)
Suzuki
('honda mitsubishi', 'NA', 1990)
('honda mitsubishi', 'NA', 1990)
Datsun
('240z', '240Z', 1973)
Plymouth
('roadrunner', 'NA', 1968)
Packard
('2 dr', 'NA', 1950)


[('Outlander Sport SE', 'Outlander Sport', 2021),
 ('Outlander PHEV LE', 'Outlander', 2021),
 ('Outlander', 'Outlander', 2020),
 ('Outlander Sport', 'Outlander Sport', 2020),
 ('Outlander', 'Outlander', 2020),
 ('Mirage', 'Mirage', 2020),
 ('Outlander SEL 4x4 gas', 'Outlander', 2020),
 ('Outlander SEL 4x4 gas', 'Outlander', 2020),
 ('Outlander SEL 4x4 gas', 'Outlander', 2020),
 ('mirage', 'Mirage', 2019),
 ('Lancer', 'Lancer', 2017),
 ('MIRAGE G4', 'Mirage', 2017),
 ('OUTLANDER SPORT', 'Outlander Sport', 2014),
 ('Lancer Evolution', 'Lancer', 2013),
 ('outlander sport es 4d', 'Outlander', 2012),
 ('LANCER', 'Lancer', 2008),
 ('LANCER', 'Lancer', 2008),
 ('eclipse gs 2dr hatchb', 'Eclipse', 2006),
 ('lancer', 'Lancer', 2006),
 ('Endeavor', 'Endeavor', 2004),
 ('lancer', 'Lancer', 2003),
 ('Sentra SR', 'Sentra', 2021),
 ('Armada SV', 'Armada', 2021),
 ('Murano Platinum', 'Murano', 2021),
 ('Altima 2.5 SR', 'Altima', 2021),
 ('Murano Platinum', 'Murano', 2021),
 ('Altima 2.5 SV', 'Altima'

In [79]:
answer

[('Outlander Sport SE', 'Outlander Sport', 2021),
 ('Outlander PHEV LE', 'Outlander', 2021),
 ('Outlander', 'Outlander', 2020),
 ('Outlander Sport', 'Outlander Sport', 2020),
 ('Outlander', 'Outlander', 2020),
 ('Mirage', 'Mirage', 2020),
 ('Outlander SEL 4x4 gas', 'Outlander', 2020),
 ('Outlander SEL 4x4 gas', 'Outlander', 2020),
 ('Outlander SEL 4x4 gas', 'Outlander', 2020),
 ('mirage', 'Mirage', 2019),
 ('Lancer', 'Lancer', 2017),
 ('MIRAGE G4', 'Mirage', 2017),
 ('OUTLANDER SPORT', 'Outlander Sport', 2014),
 ('Lancer Evolution', 'Lancer', 2013),
 ('outlander sport es 4d', 'Outlander', 2012),
 ('LANCER', 'Lancer', 2008),
 ('LANCER', 'Lancer', 2008),
 ('eclipse gs 2dr hatchb', 'Eclipse', 2006),
 ('lancer', 'Lancer', 2006),
 ('Endeavor', 'Endeavor', 2004),
 ('lancer', 'Lancer', 2003),
 ('Sentra SR', 'Sentra', 2021),
 ('Armada SV', 'Armada', 2021),
 ('Murano Platinum', 'Murano', 2021),
 ('Altima 2.5 SR', 'Altima', 2021),
 ('Murano Platinum', 'Murano', 2021),
 ('Altima 2.5 SV', 'Altima'

In [60]:
requests.get(url.format(2008), headers=headers).content

b'?({"Models":[{"model_name":"Carisma","model_make_id":"mitsubishi"},{"model_name":"Colt","model_make_id":"mitsubishi"},{"model_name":"Eclipse","model_make_id":"mitsubishi"},{"model_name":"Endeavor","model_make_id":"mitsubishi"},{"model_name":"Galant","model_make_id":"mitsubishi"},{"model_name":"Grandis","model_make_id":"mitsubishi"},{"model_name":"L 200","model_make_id":"mitsubishi"},{"model_name":"Lancer","model_make_id":"mitsubishi"},{"model_name":"Outlander","model_make_id":"mitsubishi"},{"model_name":"Pajero","model_make_id":"mitsubishi"},{"model_name":"Raider","model_make_id":"mitsubishi"},{"model_name":"Space Star","model_make_id":"mitsubishi"}]});'

In [73]:
"AAAAAAAAA 25".capitalize()

'Aaaaaaaaa 25'